This notebook checks the behaviour of the embedding space

In [1]:
import pickle
import random
import glob
import os
import sys
import scipy.sparse
import re
#import torch

import numpy as np
import pandas as pd

from annoy import AnnoyIndex
from scipy.spatial import distance

scriptpath = "../../"
sys.path.append(os.path.abspath(scriptpath))
from helpers.helpers_channels_more_300 import *

In [2]:
'''
Retrieve the array obtained by apllying the dimentionality reduction algorithm
graph_matrix: SHAPE: (channels, n_comp)

PARAMETER:
    - file_path: the path where the embedding graph is stored

RETURN: 
    - df: DataFrame representing the graph in the embedding space
'''
def create_dataframe_in_embedding_space(model_path):
    graph_matrix = torch.load(model_path)['embedding'].cpu().detach().numpy()
    df = pd.DataFrame(graph_matrix)
    df = df.rename(lambda x: 'dr'+str(x), axis='columns')
    return df

def get_dataframe_in_embedding_space(model_path):
    #graph_matrix = torch.load(model_path)['embedding'].cpu().detach().numpy()
    #df = pd.DataFrame(graph_matrix)
    #df = df.rename(lambda x: 'dr'+str(x), axis='columns')
    return pd.read_csv(model_path, compression='gzip')

In [7]:
model_path = '/dlabdata1/youtube_large/jouven/word2vec_pytorch/run_channels_more_300/CONTEXT_True_20_SUBSAMPLING_False_0.0043/models/model0.pth'
df = create_dataframe_in_embedding_space(model_path)

In [8]:
df.shape

(129616, 200)

In [9]:
df.to_csv('/dlabdata1/youtube_large/jouven/word2vec_pytorch/run_channels_more_300/CONTEXT_True_20_SUBSAMPLING_False_0.0043/models/embedding0.csv.gz', compression='gzip', index = False)


In [3]:
# Selected channels and id-index mapping
dict_channel_ind, dict_ind_channel, channels_id = filtered_channels_index_id_mapping()

### Find k closest channel using annoy library

First to check how good is the embedding space, we are going to choose a channel and it's k closest channels in the embedding space. By looking at these channels in the YouTube website, we should get similar channels if the embedding space is good.

In [4]:
channelcrawler = pd.read_csv("/dlabdata1/youtube_large/channelcrawler.csv")
channelcrawler['channel_id'] = channelcrawler['link'].str.split('/').str[-1]
channelcrawler = channelcrawler[channelcrawler['channel_id'].apply(lambda row: row in channels_id)]

In [5]:
channelcrawler['category'].unique()

array(['Film and Animation', 'Music', 'Comedy', 'Gaming',
       'Science & Technology', 'Sports', 'Entertainment', 'Education',
       'Nonprofits & Activism', 'People & Blogs', 'Howto & Style',
       'News & Politics', 'Travel & Events', 'Autos & Vehicles',
       'Pets & Animals', nan], dtype=object)

In [4]:
'''
Retrieve the array obtained by apllying the dimentinality reductin algorithm
graph_matrix: SHAPE: (channels, n_comp)

PARAMETERS:
    - df_embedding: DataFrame representing the graph in the embedding space
    - n_comp: number of components to use after the dimentionalit reduction

RETURN: The annoy index
'''
def get_annoy_index(df):
    index = AnnoyIndex(df.shape[1], "euclidean")  # Length of item vector that will be indexed
    df.apply(lambda row: index.add_item(row.name, np.array(row)), axis = 1)
    index.build(100) # 100 trees
    return index

In [5]:
def get_k_nearest_neighbors(path, ref_index_channel, k = 20):
    df = get_dataframe_in_embedding_space(path)
    index = get_annoy_index(df)
    nearest_neighbors_index = index.get_nns_by_item(ref_index_channel, k)
    nearest_neighbors_id = [dict_ind_channel[val] for val in nearest_neighbors_index]
    return nearest_neighbors_id

In [8]:
path = '/dlabdata1/youtube_large/jouven/word2vec_pytorch/run_channels_more_300/CONTEXT_True_20_SUBSAMPLING_True_0.0043/models/embedding0.csv.gz'
nearest_neighbors_id = get_k_nearest_neighbors(path, dict_channel_ind['UCMpOz2KEfkSdd5JeIJh_fxw'], k= 40)


In [9]:
nearest_neighbors = pd.DataFrame(nearest_neighbors_id, columns = ['channel_id']).merge(channelcrawler)
nearest_neighbors

,channel_id,category,join_date,link,name,subscribers,videos
0,UCMpOz2KEfkSdd5JeIJh_fxw,Howto & Style,2009-01-18,http://www.youtube.com/channel/UCMpOz2KEfkSdd5...,Shaaanxo,3215340,1422
1,UC6jgzx2g3nlbaYkd8EMweKA,Howto & Style,2010-09-07,http://www.youtube.com/channel/UC6jgzx2g3nlbaY...,Jaclyn Hill,5890000,344
2,UCbO9bltbkYwa56nZFQx6XJg,Howto & Style,2014-07-17,http://www.youtube.com/channel/UCbO9bltbkYwa56...,Manny Mua,4800000,426
3,UCKMugoa0uHpjUuq14yOpagw,Howto & Style,2009-06-09,http://www.youtube.com/channel/UCKMugoa0uHpjUu...,Laura Lee,4400000,638
4,UCzTKskwIc_-a0cGvCXA848Q,Howto & Style,2008-06-23,http://www.youtube.com/channel/UCzTKskwIc_-a0c...,NikkieTutorials,12269235,742
5,UCGwPbAQdGA3_88WBuGtg9tw,Howto & Style,2010-12-03,http://www.youtube.com/channel/UCGwPbAQdGA3_88...,grav3yardgirl,8560000,1536
6,UC21yq4sq8uxTcfgIxxyE9VQ,Howto & Style,2011-06-27,http://www.youtube.com/channel/UC21yq4sq8uxTcf...,Carli Bybel,6190000,546
7,UCkvK_5omS-42Ovgah8KRKtg,Howto & Style,2006-02-14,http://www.youtube.com/channel/UCkvK_5omS-42Ov...,jeffreestar,16000000,357
8,UC9TreTE-iXwfwQl72DzDurA,Howto & Style,2009-01-04,http://www.youtube.com/channel/UC9TreTE-iXwfwQ...,Kandee Johnson,3900000,630
9,UCF2oW5-MO8dB6ul9WH9xi0A,People & Blogs,2007-01-04,http://www.youtube.com/channel/UCF2oW5-MO8dB6u...,blndsundoll4mj,4900000,2420


In [10]:
nearest_neighbors_id = get_k_nearest_neighbors(path, dict_channel_ind['UCZyCposXwcyopaACep44maQ'], k= 40)
nearest_neighbors = pd.DataFrame(nearest_neighbors_id, columns = ['channel_id']).merge(channelcrawler)
nearest_neighbors

,channel_id,category,join_date,link,name,subscribers,videos
0,UCZyCposXwcyopaACep44maQ,Howto & Style,2011-09-29,http://www.youtube.com/channel/UCZyCposXwcyopa...,Alex Costa,2120000,543
1,UCsTcErHg8oDvUnTzoqsYeNw,Science & Technology,2010-12-21,http://www.youtube.com/channel/UCsTcErHg8oDvUn...,Unbox Therapy,15300000,1672
2,UChNN7VBxPTiNrqjUaQd9bxA,Howto & Style,2012-09-14,http://www.youtube.com/channel/UChNN7VBxPTiNrq...,Teachingmensfashion,4210000,1160
3,UC_hoQDD6zKcIqpIYLsFbBeA,People & Blogs,2013-09-22,http://www.youtube.com/channel/UC_hoQDD6zKcIqp...,Mo Vlogs,8280000,1500
4,UC5-gWZXAQqSGVfPHkA7NRiQ,Music,2008-10-02,http://www.youtube.com/channel/UC5-gWZXAQqSGVf...,Bebe Rexha,6930000,74
5,UCj34AOIMl_k1fF7hcBkD_dw,Science & Technology,2009-01-27,http://www.youtube.com/channel/UCj34AOIMl_k1fF...,EverythingApplePro,7630000,1639
6,UC1KbedtKa3d5dleFR6OjQMg,Howto & Style,2008-07-06,http://www.youtube.com/channel/UC1KbedtKa3d5dl...,alpha m.,5360000,1116
7,UCJ5v_MCY6GNUBTO8-D3XoAg,Sports,2007-05-11,http://www.youtube.com/channel/UCJ5v_MCY6GNUBT...,WWE,48400000,43421
8,UC64guZp8DXzqrIQ5OPedviw,Entertainment,2014-05-24,http://www.youtube.com/channel/UC64guZp8DXzqrI...,HoomanTV,7880000,61
9,UCRzzwLpLiUNIs6YOPe33eMg,Music,2012-10-22,http://www.youtube.com/channel/UCRzzwLpLiUNIs6...,ChainsmokersVEVO,12000000,162


In [11]:
nearest_neighbors_id = get_k_nearest_neighbors(path, dict_channel_ind['UCAql2DyGU2un1Ei2nMYsqOA'], k= 20)
nearest_neighbors = pd.DataFrame(nearest_neighbors_id, columns = ['channel_id']).merge(channelcrawler)
nearest_neighbors

,channel_id,category,join_date,link,name,subscribers,videos
0,UCAql2DyGU2un1Ei2nMYsqOA,News & Politics,2015-03-17,http://www.youtube.com/channel/UCAql2DyGU2un1E...,Donald J Trump,189000,222
1,UC1yBKRuGpC1tSM73A0ZjYjQ,News & Politics,2005-12-21,http://www.youtube.com/channel/UC1yBKRuGpC1tSM...,The Young Turks,4470000,37847
2,UCu4XcDBdnZkV6-5z2f16M0g,Film and Animation,2015-02-16,http://www.youtube.com/channel/UCu4XcDBdnZkV6-...,Real Stories,2740000,740
3,UCP6HGa63sBC7-KHtkme-p-g,News & Politics,2006-03-20,http://www.youtube.com/channel/UCP6HGa63sBC7-K...,USA TODAY,1100000,24033
4,UCBi2mrWuNuyYy4gbM6fU18Q,News & Politics,2006-08-07,http://www.youtube.com/channel/UCBi2mrWuNuyYy4...,ABC News,6640000,48869
5,UCupvZG-5ko_eiXAupbDfxWw,News & Politics,2005-10-02,http://www.youtube.com/channel/UCupvZG-5ko_eiX...,CNN,7500000,146299
6,UCqnbDFdCpuN8CMEg0VuEBqA,News & Politics,2006-10-13,http://www.youtube.com/channel/UCqnbDFdCpuN8CM...,The New York Times,2150000,9619
7,UCkH1uDkyuO9sVjSqdqBygOg,News & Politics,2013-12-04,http://www.youtube.com/channel/UCkH1uDkyuO9sVj...,CBS Los Angeles,327000,41798
8,UCNIFiHaLZkYASaWDdkC1njg,Entertainment,2006-09-20,http://www.youtube.com/channel/UCNIFiHaLZkYASa...,A&E;,3040000,6782
9,UC69uYUqvx-vw4luuX7aHNLQ,News & Politics,2014-07-11,http://www.youtube.com/channel/UC69uYUqvx-vw4l...,True Crime Daily,3250000,3830


In [12]:
nearest_neighbors_id = get_k_nearest_neighbors(path, dict_channel_ind['UC9pXxdNqCc2zjgRXSoowNNg'], k= 20)
nearest_neighbors = pd.DataFrame(nearest_neighbors_id, columns = ['channel_id']).merge(channelcrawler)
nearest_neighbors

,channel_id,category,join_date,link,name,subscribers,videos
0,UC9pXxdNqCc2zjgRXSoowNNg,News & Politics,2014-07-09,http://www.youtube.com/channel/UC9pXxdNqCc2zjg...,A Blue Dot In Texas,10600,646
1,UCk2ywidaKVmMW4QRTO2-TRg,People & Blogs,2017-09-13,http://www.youtube.com/channel/UCk2ywidaKVmMW4...,Love Withstanding,115000,101
2,UChZ0yXxxtr1nIlZdb6Txh8g,Howto & Style,2011-07-28,http://www.youtube.com/channel/UChZ0yXxxtr1nIl...,chriscook4u2,196000,372
3,UCabaQPYxxKepWUsEVQMT4Kw,Education,2013-09-29,http://www.youtube.com/channel/UCabaQPYxxKepWU...,Healthcare Triage,332000,557
4,UCbrrH1B8BIdDTTrGXApBwjQ,Entertainment,2007-10-22,http://www.youtube.com/channel/UCbrrH1B8BIdDTT...,TrueExclusives,75000,1310
5,UClSaCc5ltWsyq9qYmni-0YA,Music,2007-01-25,http://www.youtube.com/channel/UClSaCc5ltWsyq9...,V-103 Atlanta,152000,4456
6,UCS9OXp1XZh5qv5nN11GHIzQ,People & Blogs,2007-01-28,http://www.youtube.com/channel/UCS9OXp1XZh5qv5...,MadameNoire,223000,1841
7,UC9ecQ72p226Rbsm-leyxE8A,Howto & Style,2017-01-05,http://www.youtube.com/channel/UC9ecQ72p226Rbs...,BehindTheScience,284000,945
8,UCUtWNBWbFL9We-cdXkiAuJA,Howto & Style,2006-06-08,http://www.youtube.com/channel/UCUtWNBWbFL9We-...,This Old House,1240000,1633
9,UCAAmkc2sRESFoEtiKRTaTRA,News & Politics,2008-10-02,http://www.youtube.com/channel/UCAAmkc2sRESFoE...,News 5 Cleveland,138000,53493


In [13]:
nearest_neighbors_id = get_k_nearest_neighbors(path, dict_channel_ind['UCAuUUnT6oDeKwE6v1NGQxug'], k= 20)
nearest_neighbors = pd.DataFrame(nearest_neighbors_id, columns = ['channel_id']).merge(channelcrawler)
nearest_neighbors

,channel_id,category,join_date,link,name,subscribers,videos
0,UCAuUUnT6oDeKwE6v1NGQxug,People & Blogs,2006-12-06,http://www.youtube.com/channel/UCAuUUnT6oDeKwE...,TED,14800000,3105
1,UCsT0YIqwnpJCM-mx7-gSA4Q,Nonprofits & Activism,2009-06-23,http://www.youtube.com/channel/UCsT0YIqwnpJCM-...,TEDx Talks,20700000,141627
2,UCn8zNIfYAQNdrFRrr8oibKw,Entertainment,2005-12-16,http://www.youtube.com/channel/UCn8zNIfYAQNdrF...,VICE,11700000,2755
3,UC3XTzVzaHQEd30rQbuvCtTQ,Entertainment,2014-03-18,http://www.youtube.com/channel/UC3XTzVzaHQEd30...,LastWeekTonight,7240000,286
4,UCHnyfMqiRRG1u-2MsSQLbXA,Education,2010-07-21,http://www.youtube.com/channel/UCHnyfMqiRRG1u-...,Veritasium,6370000,280
5,UCpko_-a4wgz2u_DgDgd9fqA,People & Blogs,2011-08-10,http://www.youtube.com/channel/UCpko_-a4wgz2u_...,BuzzFeedVideo,19400000,6334
6,UCLXo7UDZvByw2ixzpQCufnA,News & Politics,2014-03-04,http://www.youtube.com/channel/UCLXo7UDZvByw2i...,Vox,6570000,1048
7,UCi7GJNg51C3jgmYTUwqoUXA,Comedy,2008-06-23,http://www.youtube.com/channel/UCi7GJNg51C3jgm...,Team Coco,7290000,7901
8,UC1yBKRuGpC1tSM73A0ZjYjQ,News & Politics,2005-12-21,http://www.youtube.com/channel/UC1yBKRuGpC1tSM...,The Young Turks,4470000,37847
9,UCBi2mrWuNuyYy4gbM6fU18Q,News & Politics,2006-08-07,http://www.youtube.com/channel/UCBi2mrWuNuyYy4...,ABC News,6640000,48869


In [14]:
nearest_neighbors_id = get_k_nearest_neighbors(path, dict_channel_ind['UCwTkM6CvIsYFaFiMKIKCqHw'], k= 20)
nearest_neighbors = pd.DataFrame(nearest_neighbors_id, columns = ['channel_id']).merge(channelcrawler)
nearest_neighbors

,channel_id,category,join_date,link,name,subscribers,videos
0,UCwTkM6CvIsYFaFiMKIKCqHw,Film and Animation,2005-11-27,http://www.youtube.com/channel/UCwTkM6CvIsYFaF...,James Bond 007,115000,407
1,UCzNAswnSN0rZy79clU-DRPg,Entertainment,2006-01-03,http://www.youtube.com/channel/UCzNAswnSN0rZy7...,kipkay,2630000,401
2,UCcV40gnTH9-T8yw3FHjzWZQ,People & Blogs,2007-09-19,http://www.youtube.com/channel/UCcV40gnTH9-T8y...,Philly D,1230000,1158
3,UC6wz-VSpEJZhfyAzp7LUb6g,Science & Technology,2008-02-14,http://www.youtube.com/channel/UC6wz-VSpEJZhfy...,Ty Moss,455228,797
4,UCxOD5c7H4zepc1_Q-AbJNMg,Science & Technology,2006-09-04,http://www.youtube.com/channel/UCxOD5c7H4zepc1...,SlashGear,71800,3629
5,UCCjyq_K1Xwfg8Lndy7lKMpA,Science & Technology,2005-08-07,http://www.youtube.com/channel/UCCjyq_K1Xwfg8L...,TechCrunch,436000,8270
6,UCQCJ4E2Nc9XaCL7O_VFVEjA,Music,2006-09-18,http://www.youtube.com/channel/UCQCJ4E2Nc9XaCL...,Lily Allen,831000,80
7,UCohfGoL208cZbSSlXgMNUiA,Comedy,2006-03-26,http://www.youtube.com/channel/UCohfGoL208cZbS...,BlameSociety,247000,1148
8,UCnFP0IU4gpnmcLnVzDLUtfw,Entertainment,2006-10-30,http://www.youtube.com/channel/UCnFP0IU4gpnmcL...,Blendtec's Will It B...,877000,183
9,UCPgLNge0xqQHWM5B5EFH9Cg,News & Politics,2006-11-19,http://www.youtube.com/channel/UCPgLNge0xqQHWM...,The Telegraph,678000,19104


In [15]:
nearest_neighbors_id = get_k_nearest_neighbors(path, dict_channel_ind['UCISF5OGuAtSLNF24TKTnXag'], k= 20)
nearest_neighbors = pd.DataFrame(nearest_neighbors_id, columns = ['channel_id']).merge(channelcrawler)
nearest_neighbors

,channel_id,category,join_date,link,name,subscribers,videos
0,UCISF5OGuAtSLNF24TKTnXag,Film and Animation,2013-12-17,http://www.youtube.com/channel/UCISF5OGuAtSLNF...,Magpiepony,736000,304
1,UCfR-4yZACIMRs3wNf0hel6g,Gaming,2013-06-18,http://www.youtube.com/channel/UCfR-4yZACIMRs3...,Bijuu Mike,2120000,1808
2,UC_nEHeUEVNY5ZYLRWg8KoZQ,Film and Animation,2016-11-06,http://www.youtube.com/channel/UC_nEHeUEVNY5ZY...,Wolfychu,2480000,101
3,UCzfyYtgvkx5mLy8nlLlayYg,Film and Animation,2012-09-09,http://www.youtube.com/channel/UCzfyYtgvkx5mLy...,Vivziepop,1430000,225
4,UCqg2eLFNUu3QN3dttNeOWkw,Gaming,2010-08-20,http://www.youtube.com/channel/UCqg2eLFNUu3QN3...,iHasCupquake,6620000,4261
5,UCfguohQ13Uez5UWqJbZA5Qg,Music,2012-09-09,http://www.youtube.com/channel/UCfguohQ13Uez5U...,VideoGameRapBattles,365258,93
6,UCzYfz8uibvnB7Yc1LjePi4g,Gaming,2012-08-13,http://www.youtube.com/channel/UCzYfz8uibvnB7Y...,Aphmau,4840000,2991
7,UCD_VOth7RmckN6DbmFJa__A,Entertainment,2013-08-31,http://www.youtube.com/channel/UCD_VOth7RmckN6...,The Brony Notion,256132,158
8,UCK1HgDhsRulLj-pWi8XrqIA,Gaming,2012-07-23,http://www.youtube.com/channel/UCK1HgDhsRulLj-...,Razzbowski,1160000,2101
9,UCt4qeayu0d0Yo5NHx_3xY9A,Gaming,2012-11-12,http://www.youtube.com/channel/UCt4qeayu0d0Yo5...,SirSkyward,185000,1920


In [16]:
nearest_neighbors_id = get_k_nearest_neighbors(path, dict_channel_ind['UC_aP7p621ATY_yAa8jMqUVA'], k= 20)
nearest_neighbors = pd.DataFrame(nearest_neighbors_id, columns = ['channel_id']).merge(channelcrawler)
nearest_neighbors

,channel_id,category,join_date,link,name,subscribers,videos
0,UC_aP7p621ATY_yAa8jMqUVA,Science & Technology,2013-04-05,http://www.youtube.com/channel/UC_aP7p621ATY_y...,NASA Video,391000,8232
1,UCvGMGQC8gNkd4gwxSbABIlw,Education,2007-04-22,http://www.youtube.com/channel/UCvGMGQC8gNkd4g...,Quirkology,2149593,98
2,UCA6aJAT9rH8vRwWGiGr2iqQ,Autos & Vehicles,2011-03-29,http://www.youtube.com/channel/UCA6aJAT9rH8vRw...,Cargospotter,486920,619
3,UCtM5z2gkrGRuWd0JQMx76qA,Science & Technology,2007-01-14,http://www.youtube.com/channel/UCtM5z2gkrGRuWd...,bigclivedotcom,555495,1509
4,UC_7aK9PpYTqt08ERh1MewlQ,Science & Technology,2015-04-03,http://www.youtube.com/channel/UC_7aK9PpYTqt08...,3D Printing Nerd,340000,713
5,UCc4iogeOUXNw1RZSNTOlMeg,News & Politics,2010-07-21,http://www.youtube.com/channel/UCc4iogeOUXNw1R...,Corey Gil-Shuster,139906,833
6,UCAzyupPG4vdo8jd9nJ13LAw,Howto & Style,2009-12-30,http://www.youtube.com/channel/UCAzyupPG4vdo8j...,Barry Lewis,866000,1470
7,UCzofNVHFCdD_4Jxs5dVqtAA,Entertainment,2006-11-07,http://www.youtube.com/channel/UCzofNVHFCdD_4J...,TEAMSUPERTRAMP,970000,486
8,UCVhibwHk4WKw4leUt6JfRLg,Sports,2010-03-24,http://www.youtube.com/channel/UCVhibwHk4WKw4l...,SPORTSNET,341000,19675
9,UCzhnStmSSpfXdDkvRjOjo9A,People & Blogs,2013-07-02,http://www.youtube.com/channel/UCzhnStmSSpfXdD...,Matt & Amanda,954898,1281


### Channels selected over the whole comments dataset
We randomly choose 10 000 users over the dataset.
For each user, we then pick two channels at random in the set of channels this user commented in.

In [6]:
with open("/dlabdata1/youtube_large/jouven/channels_more_300/channels_tuple_user_walk.pkl",'rb') as f:
     channels_tuple = pickle.load(f)
f.close()

#### Random walk

In [7]:
def get_random_walk(df_embedding):
    with open("/dlabdata1/youtube_large/jouven/channels_more_300/channels_tuple_random_walk.pkl",'rb') as f:
         random_walk_channels = pickle.load(f)
    f.close()
    random_walk_distance = 0
    for val in random_walk_channels:
        random_walk_distance += distance.euclidean(df_embedding.iloc[val[0]], df_embedding.iloc[val[1]])
    return random_walk_distance

def get_random_walk_new(df_embedding):
    with open("/dlabdata1/youtube_large/jouven/channels_more_300/channels_tuple_random_walk_modified.pkl",'rb') as f:
         random_walk_channels = pickle.load(f)
    f.close()
    random_walk_distance = 0
    for val in random_walk_channels:
        random_walk_distance += distance.euclidean(df_embedding.iloc[val[0]], df_embedding.iloc[val[1]])
    return random_walk_distance

#### Compute metrics: users walk distance and relative nearest neighbor ranking 

In this section we want to measure the euclidian distance of a user walk compared to a random walk.

user walk: Euclidean distance in the embedding space between two randomly channels of a user.
random walk: Euclidean distance in the embedding space between two randomly channels.
position: Position of a channel taken from user u relatively of another channel taken from the same user in terms of its nearest neighbor ranking.


In [8]:
'''
Get the position of ref_channel relative to second_channel in terms of its nearest neighbors ranking.
PARAMETER:
    - ref_channel: The reference channel on which wwe compute it's k nearest neighbor
    - second_channel: The channel where we compute it's ranking relatively to ref_channel
    - dist: Euclidean distance between ref_channel and second_channel
    - index: annoy index
    - df_embedding: DataFrame representing the embedding space

RETURN: The position of second_channel relatively to ref_channel in terms of it's ranking

'''
def get_ranking_position_between_channels(ref_channel, second_channel, index, df_embedding):
    
    nearest_neighbors_index = index.get_nns_by_item(ref_channel, len(df_embedding), search_k = 100000000)
    dist_k_th_nearest = distance.euclidean(df_embedding.iloc[ref_channel], 
                                           df_embedding.iloc[nearest_neighbors_index[len(nearest_neighbors_index)-1]])
    for i in range(0, len(nearest_neighbors_index)):
        if nearest_neighbors_index[i] == second_channel:
            return i
    

In [9]:
users_walk_tab = []
users_walk_tab_new = []
ranking_position_tab = []

len_random_set = len(channels_tuple)
len_embedding = len(channels_id)

#path = '/dlabdata1/youtube_large/jouven/channel_embedding/limited_normalized_50/'
#files = glob.glob(path + '*.npz') 
model_1 = '/dlabdata1/youtube_large/jouven/word2vec_pytorch/run_channels_more_300/CONTEXT_True_20_SUBSAMPLING_True_0.0043/models/embedding0.csv.gz'

files = [model_1]
for file in files: 
    print('file ', file)
    df_embedding = get_dataframe_in_embedding_space(file)
    n_comp = df_embedding.shape[1]
    print('n_comp ', n_comp)
    random_walk_distance = get_random_walk(df_embedding)
    random_walk_distance_new = get_random_walk_new(df_embedding)
    index = get_annoy_index(df_embedding)
    users_walk = 0
    ranking_position = 0

    for ref_channel, second_channel in channels_tuple:
        users_walk += distance.euclidean(df_embedding.iloc[ref_channel], df_embedding.iloc[second_channel])
        ranking_position += get_ranking_position_between_channels(ref_channel, second_channel, index, df_embedding)
    
    users_walk_tab.append(users_walk/random_walk_distance)
    users_walk_tab_new.append(users_walk/random_walk_distance_new)
    ranking_position_tab.append(ranking_position/(len_random_set*len_embedding))


file  /dlabdata1/youtube_large/jouven/word2vec_pytorch/run_channels_more_300/CONTEXT_True_20_SUBSAMPLING_True_0.0043/models/embedding0.csv.gz
n_comp  200


In [10]:
users_walk_tab

[0.4297293346660091]

In [11]:
ranking_position_tab

[0.09066697269884376]

In [12]:
users_walk_tab_new

[1.8439751582788533]

In [ ]:
'''
Plot the results obtained when computing the metrics
'''
import plotly.graph_objects as go

components_name = ['Features dim 50', 'Features dim 100', 'Features dim 200', 'Features dim 500']
colors = ['b', 'g,', 'r', 'c']

fig = go.Figure()

for i in range(len(components_name)):
    fig.add_trace(go.Scatter(
        x=ranking_position_tab,
        y=users_walk_tab,
        marker=dict(color=colors[i], size=12),
        mode="markers",
        name=components_name[i],
    ))

fig.update_layout(title="Gender Earnings Disparity",
                  xaxis_title="Ranking position",
                  yaxis_title="Users walk distance")

fig.show()